In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd 
import numpy as np
from preprocessing.cleaning_utils import *
from functools import partial
import torch 

from datasets import load_dataset
from datasets import Value, ClassLabel, Features, DatasetDict
import transformers
from transformers import AutoTokenizer, AutoModel, AutoModelForMaskedLM, AutoModelForSequenceClassification
from transformers import logging
from transformers import TrainingArguments, Trainer

logging.set_verbosity_warning()

In [3]:
mimic_dir = "/home/vs428/project/MIMIC/files/mimiciii/1.4/"
n2c2_dir = "/home/vs428/project/n2c2/2022/N2C2-AP-Reasoning/"

In [4]:
samples = pd.read_csv(n2c2_dir + "n2c2_sample.csv")
samples_raw = pd.read_csv(n2c2_dir + "n2c2_sample_raw.csv")

In [ ]:
display(samples_raw), display(samples)

In [ ]:
samples['Relation'].value_counts().index

In [ ]:
ids = samples['ROW ID'].unique()
ids2 = samples['HADM ID'].unique()
ids, ids2

In [ ]:
notes = pd.read_csv(mimic_dir + "NOTEEVENTS.csv")

In [ ]:
notes.columns

In [ ]:
notes[notes['ROW_ID'] == 668984]['TEXT'].values[0][8402:9512]

In [ ]:
samples_raw

In [69]:

classes = ['Not Relevant', 'Neither', 'Indirect', 'Direct']
features = Features({
    'ROW ID':Value("int64"),
    'HADM ID':Value("int64"),
    'Assessment':Value("string"),
    'PlanSubsection':Value("string"),
    "Relation":Value("string")
}) 
dataset = load_dataset("csv", data_files=n2c2_dir + "n2c2_sample_raw.csv", 
                       features=features)
dataset = dataset.class_encode_column("Relation")
dataset = dataset.rename_column("Relation", "label")

Using custom data configuration default-b4d529c722a7e397
Reusing dataset csv (/home/vs428/.cache/huggingface/datasets/csv/default-b4d529c722a7e397/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached processed dataset at /home/vs428/.cache/huggingface/datasets/csv/default-b4d529c722a7e397/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-f79a282eb4f7598e.arrow
Loading cached processed dataset at /home/vs428/.cache/huggingface/datasets/csv/default-b4d529c722a7e397/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-9f42d8cb8e4b4c85.arrow


In [70]:
# split dataset: 80% train, 10% validation, 10% test 
train_testvalid = dataset['train'].train_test_split(test_size=0.2)
test_valid = train_testvalid['test'].train_test_split(test_size=0.5)
split_dataset = DatasetDict({
    'train': train_testvalid['train'],
    'valid': test_valid['train'],
    'test': test_valid['test'],
})



Loading cached split indices for dataset at /home/vs428/.cache/huggingface/datasets/csv/default-b4d529c722a7e397/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-ebd0b8dfe9567602.arrow and /home/vs428/.cache/huggingface/datasets/csv/default-b4d529c722a7e397/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-e4255d75b8e7b04e.arrow
Loading cached split indices for dataset at /home/vs428/.cache/huggingface/datasets/csv/default-b4d529c722a7e397/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-9dfdc178874284ca.arrow and /home/vs428/.cache/huggingface/datasets/csv/default-b4d529c722a7e397/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-69125e18c294ec55.arrow


In [71]:
split_dataset

DatasetDict({
    train: Dataset({
        features: ['ROW ID', 'HADM ID', 'Assessment', 'PlanSubsection', 'label'],
        num_rows: 58
    })
    valid: Dataset({
        features: ['ROW ID', 'HADM ID', 'Assessment', 'PlanSubsection', 'label'],
        num_rows: 7
    })
    test: Dataset({
        features: ['ROW ID', 'HADM ID', 'Assessment', 'PlanSubsection', 'label'],
        num_rows: 8
    })
})

In [72]:
bio_clinicalbert_tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/emilyalsentzer/Bio_ClinicalBERT/resolve/main/config.json from cache at /home/vs428/.cache/huggingface/transformers/dc6d60ebe42d83e1479ce0d473758bb3586763ff6c4c814bda5321acf856bd64.b74d0770929e519c6d193d16b6874051ae549f5c8c228903a48e59d36260466b
Model config BertConfig {
  "_name_or_path": "emilyalsentzer/Bio_ClinicalBERT",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.17.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading file https:/

In [73]:
split_dataset = split_dataset.map(lambda examples: bio_clinicalbert_tokenizer(examples['Assessment'], examples['PlanSubsection'],
                                    max_length=768, 
                                    padding=True,
                                    truncation=True,
                                    verbose=True), batched=True)

Loading cached processed dataset at /home/vs428/.cache/huggingface/datasets/csv/default-b4d529c722a7e397/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-748ccbd7ca79ff8a.arrow


  0%|          | 0/1 [00:00<?, ?ba/s]

Loading cached processed dataset at /home/vs428/.cache/huggingface/datasets/csv/default-b4d529c722a7e397/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-3a1406a24f68e733.arrow


In [74]:
split_dataset['valid'][4]#['input_ids']

{'ROW ID': 430757,
 'HADM ID': 113206,
 'Assessment': 'Assessment and Plan\n   79 yo F with COPD, DM, HTN, HLP, morbid obesity, who is transferred\n   from an OSH for worsening SOB found to have a large pericardial\n   effusion with tamponade physiology on ECHO- unable to perform\n   pericardiocentesis yesterday in cath lab due to position of pericardial\n   effusion.',
 'PlanSubsection': 'ACCESS: PIV',
 'label': 3,
 'input_ids': [101,
  8670,
  1105,
  2197,
  5899,
  26063,
  175,
  1114,
  9947,
  1181,
  117,
  173,
  1306,
  117,
  177,
  1204,
  1179,
  117,
  177,
  1233,
  1643,
  117,
  182,
  1766,
  14598,
  184,
  27655,
  117,
  1150,
  1110,
  3175,
  1121,
  1126,
  184,
  2737,
  1111,
  4146,
  3381,
  20295,
  1276,
  1106,
  1138,
  170,
  1415,
  1679,
  4578,
  16936,
  1348,
  174,
  3101,
  17268,
  1114,
  27629,
  24729,
  1605,
  2007,
  25445,
  1113,
  16278,
  118,
  3372,
  1106,
  3870,
  1679,
  4578,
  16936,
  13335,
  23771,
  4863,
  8128,
  1107,
  

In [75]:
# split_dataset['valid'][4]

In [76]:
# bio_clinicalbert_tokenizer.decode(split_dataset['valid'][1]['input_ids']), split_dataset['valid'][1]['token_type_ids']

In [77]:
split_dataset['train'].set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
dataloader = torch.utils.data.DataLoader(split_dataset['train'], batch_size=16)
test = next(iter(dataloader))

In [78]:
test_x = []
for x in iter(dataloader):
    test_x.append(x)
# test = next(iter(dataloader))
# test['input_ids'].shape


In [79]:
test_x[2]['input_ids'].shape

torch.Size([16, 350])

# Train Model

In [80]:
bio_clinicalbert_model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT", num_labels=4)

loading configuration file https://huggingface.co/emilyalsentzer/Bio_ClinicalBERT/resolve/main/config.json from cache at /home/vs428/.cache/huggingface/transformers/dc6d60ebe42d83e1479ce0d473758bb3586763ff6c4c814bda5321acf856bd64.b74d0770929e519c6d193d16b6874051ae549f5c8c228903a48e59d36260466b
Model config BertConfig {
  "_name_or_path": "emilyalsentzer/Bio_ClinicalBERT",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "

In [81]:
split_dataset['train']

Dataset({
    features: ['ROW ID', 'HADM ID', 'Assessment', 'PlanSubsection', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 58
})

In [82]:
training_args = TrainingArguments(output_dir="test_trainer", 
                                  evaluation_strategy="epoch",
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [83]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

In [84]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [85]:
trainer = Trainer(
    model=bio_clinicalbert_model,
    args=training_args,
    train_dataset=split_dataset['train'],
    eval_dataset=split_dataset['valid'],
    compute_metrics=compute_metrics,
)

In [86]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertModel.forward` and have been ignored: HADM ID, Assessment, PlanSubsection, ROW ID. If HADM ID, Assessment, PlanSubsection, ROW ID are not expected by `BertModel.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 58
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 24


TypeError: forward() got an unexpected keyword argument 'labels'

In [91]:
bio_clinicalbert_model.forward()

ValueError: You have to specify either input_ids or inputs_embeds